In [2]:
import pandas as pd
import sklearn as skl
import xgboost as xgb
from datetime import timedelta
from sklearn.metrics import explained_variance_score

cols=['device_language','ref_type','wifi','carrier', 'device_brand',
      'device_os_version','attributed','ref_hash','ref_type',
      'device_model','device_city','date']
events_csv = pd.read_csv('./data/events.csv',usecols=cols)

events_csv['date'] = pd.to_datetime(events_csv['date'])
events_csv.head()

,date,ref_type,ref_hash,attributed,device_os_version,device_brand,device_model,device_city,carrier,wifi,device_language
0,2019-04-20 01:42:49.120,1891515180541284343,5857744372586891366,False,NaN,NaN,4.318294e+18,NaN,NaN,False,3.301378e+18
1,2019-04-20 01:42:49.340,1891515180541284343,7642521036780133571,False,NaN,NaN,NaN,NaN,NaN,False,NaN
2,2019-04-20 01:42:49.365,1891515180541284343,2548841562898283198,False,NaN,NaN,NaN,NaN,NaN,False,NaN
3,2019-04-20 01:42:51.438,1891515180541284343,609402887625919085,False,NaN,NaN,NaN,NaN,NaN,False,NaN
4,2019-04-20 01:42:51.838,1891515180541284343,9114651763556439823,False,NaN,NaN,NaN,NaN,NaN,False,NaN


In [3]:
events = events_csv
events.fillna(value=0, inplace = True)
events['attributed'] = events['attributed'].apply(lambda x: 1 if x == True else 0)
events['wifi'] = events['attributed'].apply(lambda x: 1 if x == True else 0)
events['ref_type'] = events['ref_type'].apply(lambda x: 1 if x == 1891515180541284343 else 0)
events['n_events'] = 1

In [4]:
def generar_ventanas(df, str_columna_fecha):
    FECHA_INICIO = pd.to_datetime(df[str_columna_fecha].min().strftime('%Y-%m-%d'))
    FECHA_FIN = pd.to_datetime(df[str_columna_fecha].max().strftime('%Y-%m-%d'))

    FECHA_UN_DIA = FECHA_INICIO + timedelta(days=1)
    FECHA_DOS_DIAS = FECHA_INICIO + timedelta(days=2)
    FECHA_TRES_DIAS = FECHA_INICIO + timedelta(days=3)
    FECHA_CUATRO_DIAS = FECHA_INICIO + timedelta(days=4)
    FECHA_CINCO_DIAS = FECHA_INICIO + timedelta(days=5)
    FECHA_SEIS_DIAS = FECHA_INICIO + timedelta(days=6)
    FECHA_SIETE_DIAS = FECHA_INICIO + timedelta(days=7)
    FECHA_OCHO_DIAS = FECHA_INICIO + timedelta(days=8)
    FECHA_NUEVE_DIAS = FECHA_INICIO + timedelta(days=9)

    ventana_uno = df.loc[(df[str_columna_fecha] < FECHA_TRES_DIAS)]
    ventana_uno = ventana_uno.loc[(df[str_columna_fecha] > FECHA_INICIO)]

    ventana_dos = df.loc[(df[str_columna_fecha] < FECHA_CUATRO_DIAS)]
    ventana_dos = ventana_dos.loc[(df[str_columna_fecha] > FECHA_UN_DIA)]

    ventana_tres = df.loc[(df[str_columna_fecha] < FECHA_CINCO_DIAS)]
    ventana_tres = ventana_tres.loc[(df[str_columna_fecha] > FECHA_DOS_DIAS)]
    
    ventana_cuatro = df.loc[(df[str_columna_fecha] < FECHA_SEIS_DIAS)]
    ventana_cuatro = ventana_cuatro.loc[(df[str_columna_fecha] > FECHA_TRES_DIAS)]
    
    ventana_cinco = df.loc[(df[str_columna_fecha] < FECHA_SIETE_DIAS)]
    ventana_cinco = ventana_cinco.loc[(df[str_columna_fecha] > FECHA_CUATRO_DIAS)]
    
    ventana_seis = df.loc[(df[str_columna_fecha] < FECHA_OCHO_DIAS)]
    ventana_seis = ventana_seis.loc[(df[str_columna_fecha] > FECHA_CINCO_DIAS)]
    
    ventana_siete = df.loc[(df[str_columna_fecha] < FECHA_NUEVE_DIAS)]
    ventana_siete = ventana_cinco.loc[(df[str_columna_fecha] > FECHA_SEIS_DIAS)]
    
    return ventana_uno, ventana_dos, ventana_tres, ventana_cuatro, ventana_cinco, ventana_seis, ventana_siete

ventana1, ventana2, ventana3, ventanta4, ventana5, ventana6, ventana7 = generar_ventanas(events,'date')

In [5]:
ventanas = (ventana1, ventana2, ventana3, ventanta4, ventana5, ventana6, ventana7)
ev_f = []
for ventana in ventanas:
    ev = ventana.groupby(['ref_hash']).agg({'attributed':'mean','wifi':'mean','n_events':'sum',
                                                       'device_os_version':'max', 'device_brand':'max', 'device_model':'max',
                                                                          'device_city':'max','device_language':'max'})
    ev.reset_index(inplace=True)
    ev_f.append(ev)

In [7]:
ev1 = ev_f[0]
ev2 = ev_f[1]
ev3 = ev_f[2]
ev4 = ev_f[3]
ev5 = ev_f[4]
ev6 = ev_f[5]
ev7 = ev_f[6]

ev1.to_csv('./data/features/events/ev1.csv', index=False)
ev2.to_csv('./data/features/events/ev2.csv', index=False)
ev3.to_csv('./data/features/events/ev3.csv', index=False)
ev4.to_csv('./data/features/events/ev4.csv', index=False)
ev5.to_csv('./data/features/events/ev5.csv', index=False)
ev6.to_csv('./data/features/events/ev6.csv', index=False)
ev7.to_csv('./data/features/events/ev7.csv', index=False)

ev1.head()

,ref_hash,device_os_version,wifi,device_brand,n_events,attributed,device_city,device_language,device_model
0,41863526108385,0.000000e+00,0.0,6.115026e+18,88,0.0,0.000000e+00,6.977049e+18,1.658417e+18
1,161514654074162,0.000000e+00,0.0,2.208835e+18,8,0.0,0.000000e+00,6.977049e+18,6.684966e+18
2,186034136943920,0.000000e+00,0.0,0.000000e+00,13,0.0,0.000000e+00,3.301378e+18,7.247662e+18
3,360710529886978,7.391844e+18,0.0,6.115026e+18,3,0.0,4.577311e+18,3.301378e+18,5.660121e+18
4,365882020742330,0.000000e+00,0.0,3.083059e+17,57,0.0,0.000000e+00,6.977049e+18,1.805456e+18
